In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

import numpy as np
import json
import os
import math
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye.xml')

In [2]:
def try_get_people(try_num_images):
    
    num_images = 0
    step = 0
    
    while(num_images <= try_num_images):
        url="https://www.google.com.br/search?ei=R2a3WoHnEcOrwATmprbwCw&yv=2&q=face+people&tbm=isch&start=%s" % step
        headers={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}

        response = urlopen(Request(url, headers=headers))
        soup = BeautifulSoup(response, 'html.parser')

        images = []
        for a in soup.find_all('div', {'class': 'rg_meta'}):
            link, Type = json.loads(a.text)['ou'], json.loads(a.text)['ity']
            images.append((link, Type))

        for _, (image, Type) in enumerate(images[0:100]):
            try:
                req = Request(image, headers=headers)
                raw_image = urlopen(req).read()

                raw_image_transf = np.fromstring(raw_image, dtype='uint8')

                cv2_image = cv2.imdecode(raw_image_transf, cv2.IMREAD_UNCHANGED)

                cv2_image_resized = cv2.resize(cv2_image, (300, 300)) 
                cv2_image_resized_without_eyes = cv2_image_resized.copy()

                cv2_image_resized_gray = cv2.cvtColor(cv2_image_resized, 0)
                faces = face_cascade.detectMultiScale(cv2_image_resized_gray, 1.3, 5)

                accept = False
                for (x, y, w, h) in faces:
                    face_gray = cv2_image_resized_gray[y:y+h, x:x+w]

                    eyes = eye_cascade.detectMultiScale(face_gray)
                    for (ex, ey, ew, eh) in eyes:
                        accept = True
                        cv2_image_resized_without_eyes[y+ey:y+ey+eh, x+ex:x+ex+ew] = 0

                if accept:
                    num_images += 1
                    cv2.imwrite('data/train/p_'+str(num_images)+'.jpg', cv2_image_resized)
                    cv2.imwrite('data/train/p_'+str(num_images)+'_without_eyes.jpg', cv2_image_resized_without_eyes)

            except Exception as e:
                print('could not load : ' + image)
                print(e)    
        step += 100

In [4]:
try_get_people(1000)

KeyboardInterrupt: 